In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/07/02 17:13:31 WARN Utils: Your hostname, Yokis-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.93 instead (on interface en0)
24/07/02 17:13:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/02 17:13:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/02 17:13:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/07/02 17:13:32 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/07/02 17:13:32 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/07/02 17:13:32 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/07/02 17:13:32 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting 

In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

```
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [3]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [4]:
from datetime import datetime

In [5]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [6]:
rows = rdd.take(10)
row = rows[0]

In [7]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 10, 20, 5, 54), PULocationID=129, total_amount=8.51)

In [8]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [9]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [10]:
from collections import namedtuple

In [11]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [12]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [13]:
from pyspark.sql import types

In [14]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [15]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [16]:
df_result.write.parquet('tmp/green-revenue')

24/07/02 17:13:44 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
24/07/02 17:15:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/07/02 17:15:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/07/02 17:15:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/07/02 17:15:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/07/02 17:15:48 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Sc

In [17]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [18]:
import pandas as pd

In [19]:
rows = duration_rdd.take(10)

In [20]:
df = pd.DataFrame(rows, columns=columns)

In [21]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [22]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [23]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [24]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [25]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
| 4.050000000000001|
|             13.45|
|             65.55|
|10.649999999999999|
|              4.45|
|               4.4|
|             11.25|
|              4.55|
|             13.45|
|             26.75|
|               2.9|
|             39.15|
|               2.5|
|               4.5|
|119.39999999999999|
|               9.8|
|               0.0|
|7.1499999999999995|
|              4.25|
|             12.05|
+------------------+
only showing top 20 rows

